## _The goal of this code is for providing generalizable, useful, working functions intented to aid in the creation of developmental indices on any gene expression data (preferably a dataset that has already been adjusted to appropriate values i.e. RPKM/FPKM/TPM)_

#### 1) This first step is to scale all of the expression (whatever level it may be in - as long as it is normalized) to values between 0 and 1 to reduce bias between genes

In [1]:
## importing the necessary packages
import numpy as np
import pandas as pd
import scipy.stats as stats

In [21]:
df = pd.read_csv('C:\\Users\\Ben\\Dropbox\\bilbo_lab_spr2020\\microglia-seq_website\\microglia-seq\\mdi_w_rpkm\\GSE99622_hanamsagar2017_tpm_unmelted_v2.csv', header = [0, 1, 2, 3, 4])

In [22]:
genes = df.iloc[:, df.columns.get_level_values(4) == 'gene'].values.flatten()

In [23]:
genes

array(['Zfp85-rs1', 'Snx7', '1700034O15Rik', ..., 'Sox1', 'Mettl2', 'Ttl'],
      dtype=object)

In [24]:
df.set_index(genes, inplace = True)
df.drop(df.columns[0], axis = 1, inplace = True)

In [25]:
### (expression value of the gene in a sample—minimum expression for the gene across all samples)/(maximum expression for the gene across all
### samples—minimum expression for the gene across all samples),
### scale all values so that they add equal weight to the index

def scale_expression(df):    
    
    ## creating a copy of the input dataframe to scale the expression on
    scaled_expression_df = df.copy()
    
    ## iterating through the rows of the dataframe to scale expression of each gene (by row)
    for row in range(len(scaled_expression_df)):
        scaled_expression_df.iloc[row] = (scaled_expression_df.iloc[row] - np.min(scaled_expression_df.iloc[row])) / np.max(scaled_expression_df.iloc[row] - np.min(scaled_expression_df.iloc[row]))        
    
    return scaled_expression_df

In [26]:
%%time

df = scale_expression(df)

Wall time: 20.7 s


In [27]:
df

E18                                        \
                          Female               Male             Female   
                             SAL                SAL                SAL   
                         F_E18 1            M_E18 1            F_E18 3   
              Unnamed: 1_level_4 Unnamed: 2_level_4 Unnamed: 3_level_4   
Zfp85-rs1               0.307701           0.947800           0.444803   
Snx7                    0.588140           0.368395           0.568197   
1700034O15Rik                NaN                NaN                NaN   
Dnajc18                 0.797903           0.653250           0.657849   
Lce1i                        NaN                NaN                NaN   
...                          ...                ...                ...   
Cdkn2aip                0.747043           0.552209           0.538767   
Phf20                   0.852279           0.625225           0.638040   
Sox1                    0.596204           0.660740           0.109442   
Mettl2                  0.732807           0.692819           0.577084   
Ttl                     0.979584           0.867743           0.731167   

                                                P14                     \
                            Male             Female                      
                             SAL                SAL                      
                         M_E18 4            F_P14 1            F_P14 2   
              Unnamed: 4_level_4 Unnamed: 5_level_4 Unnamed: 6_level_4   
Zfp85-rs1               0.000000           0.225971           0.195132   
Snx7                    1.000000           0.181462           0.219422   
1700034O15Rik                NaN                NaN                NaN   
Dnajc18                 0.000000           0.111909           0.408285   
Lce1i                        NaN                NaN                NaN   
...                          ...                ...                ...   
Cdkn2aip                0.000000           0.155256           0.379097   
Phf20                   0.000000           0.496946           0.497212   
Sox1                    0.042407           0.020850           0.021605   
Mettl2                  0.000000           0.255052           0.332333   
Ttl                     0.101719           0.317378           0.270411   

                                                                        \
                                                                         
                                                                         
                         F_P14 3            F_P14 4            F_P14 5   
              Unnamed: 7_level_4 Unnamed: 8_level_4 Unnamed: 9_level_4   
Zfp85-rs1               0.043254           0.165641           0.178076   
Snx7                    0.346333           0.241872           0.676114   
1700034O15Rik                NaN                NaN                NaN   
Dnajc18                 0.629233           0.236698           0.488088   
Lce1i                        NaN                NaN                NaN   
...                          ...                ...                ...   
Cdkn2aip                0.334960           0.341667           0.249145   
Phf20                   0.461477           0.410191           0.361023   
Sox1                    0.031927           0.003057           0.012323   
Mettl2                  0.308777           0.342049           0.053049   
Ttl                     0.364505           0.483900           0.200087   

                                   ...           P60 + LPS  \
                                   ...              Female   
                                   ...                 LPS   
                          F_P14 6  ...         F_P60_LPS 3   
              Unnamed: 10_level_4  ... Unnamed: 51_level_4   
Zfp85-rs1                0.000000  ...            0.625977   
Snx7                     0.452090  ...            0.122795   
1700034O15Rik                 NaN  ...  

In [28]:
### just a quick function for cleaning data and making sure you don't have and unexpressed genes

def drop_unexpressed_genes(df):

    df.dropna(inplace = True)
    
    return df

In [29]:
df = drop_unexpressed_genes(df)

#### 2) This next step is to identify which genes make up the index. To do this, you must give it the scaled dataframe and column labels for the two ages you would like to create the index off of

In [30]:
def identify_significant_genes(df, young, old):

    for gene, series in young.iterrows():
        if np.mean(young.loc[gene]) == 0:
            young.drop(gene, inplace = True)

    for gene, series in old.iterrows():
        if np.mean(old.loc[gene]) == 0:
            old.drop(gene, inplace = True)

    pvals = np.zeros(shape = len(df))
    sig = np.zeros(shape = len(df))
    logdiff = np.zeros(shape = len(df))
    row = -1

    for gene, series in df.iterrows():
        row = row + 1
        if gene in young.index:
            if gene in old.index:
                pvals[row] = stats.ttest_ind(young.loc[gene], old.loc[gene])[1]
                sig[row] = stats.ttest_ind(young.loc[gene], old.loc[gene])[0]
                logdiff[row] = np.log2(np.mean(old.loc[gene])/np.mean(young.loc[gene]))
            else:
                pvals[row] = np.NAN
                sig[row] = np.NAN
                logdiff[row] = np.NAN
        else:
            pvals[row] = np.NAN
            sig[row] = np.NAN
            logdiff[row] = np.NAN


    df['pvals'] = pvals
    df['sig'] = sig
    df['logdiff'] = logdiff

    direction = [0] * len(df)
    for row in range(len(df)):
        if df['pvals'][row] < 0.05:
            if df['logdiff'][row] > 0:
                direction[row] = 'UP'
            else:
                direction[row] = 'DOWN'
        else:
            direction[row] = 'N/A'
    df['direction'] = direction


    return df

In [31]:
## these lines are different based on how you have indexind the samples in your dataframe. You may need to adjust accordingly
young = df['E18']
old = df['P60']
old = old.iloc[:, old.columns.get_level_values(1)=='SAL']

In [32]:
%%time

df = identify_significant_genes(df, young, old)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Wall time: 1min 21s


#### 4) This step is meant to drop the genes in the dataframe that were not developmentally regulated. This will just make it easier to move forward with the analysis

In [33]:
df

E18                                        \
                      Female               Male             Female   
                         SAL                SAL                SAL   
                     F_E18 1            M_E18 1            F_E18 3   
          Unnamed: 1_level_4 Unnamed: 2_level_4 Unnamed: 3_level_4   
Zfp85-rs1           0.307701           0.947800           0.444803   
Snx7                0.588140           0.368395           0.568197   
Dnajc18             0.797903           0.653250           0.657849   
Ipo11               0.853112           0.442486           0.719860   
Mfsd2a              0.712312           0.576530           0.319268   
...                      ...                ...                ...   
Cdkn2aip            0.747043           0.552209           0.538767   
Phf20               0.852279           0.625225           0.638040   
Sox1                0.596204           0.660740           0.109442   
Mettl2              0.732807           0.692819           0.577084   
Ttl                 0.979584           0.867743           0.731167   

                                            P14                     \
                        Male             Female                      
                         SAL                SAL                      
                     M_E18 4            F_P14 1            F_P14 2   
          Unnamed: 4_level_4 Unnamed: 5_level_4 Unnamed: 6_level_4   
Zfp85-rs1           0.000000           0.225971           0.195132   
Snx7                1.000000           0.181462           0.219422   
Dnajc18             0.000000           0.111909           0.408285   
Ipo11               0.000000           0.208482           0.398846   
Mfsd2a              0.210441           0.216490           0.414484   
...                      ...                ...                ...   
Cdkn2aip            0.000000           0.155256           0.379097   
Phf20               0.000000           0.496946           0.497212   
Sox1                0.042407           0.020850           0.021605   
Mettl2              0.000000           0.255052           0.332333   
Ttl                 0.101719           0.317378           0.270411   

                                                                    \
                                                                     
                                                                     
                     F_P14 3            F_P14 4            F_P14 5   
          Unnamed: 7_level_4 Unnamed: 8_level_4 Unnamed: 9_level_4   
Zfp85-rs1           0.043254           0.165641           0.178076   
Snx7                0.346333           0.241872           0.676114   
Dnajc18             0.629233           0.236698           0.488088   
Ipo11               0.636584           0.382300           0.502430   
Mfsd2a              0.639350           0.250279           0.272967   
...                      ...                ...                ...   
Cdkn2aip            0.334960           0.341667           0.249145   
Phf20               0.461477           0.410191           0.361023   
Sox1                0.031927           0.003057           0.012323   
Mettl2              0.308777           0.342049           0.053049   
Ttl                 0.364505           0.483900           0.200087   

                               ...           P60 + LPS                 P60  \
                               ...                Male                Male   
                               ...                 LPS                 SAL   
                      F_P14 6  ...         M_P60_LPS 6         M_P60_Sal 1   
          Unnamed: 10_level_4  ... Unnamed: 55_level_4 Unnamed: 56_level_4   
Zfp85-rs1            0.000000  ...            0.806224            0.246272   
Snx7                 0.452090  ...            0.160842            0.294345   
Dnajc18              0.484321  ...            0.239025            0.592543   
Ipo11                0.789239  ...    

In [34]:
def remove_unsignificant_rows(df):
    for gene, series in df.iterrows():
        if df['direction'][gene] == 'N/A':
            df.drop(gene, inplace = True)  
    df.reset_index(inplace=True)
    df.rename(columns = {'index' : 'gene'}, inplace = True)
    return df

In [35]:
%%time

df2 = remove_unsignificant_rows(df)

Wall time: 1min 54s


In [41]:
df2.rename(columns = {'index' : 'gene'}, inplace = True)

#### 5) This step allows you to extract all of the information for the index genes into a standalone dataframe

In [47]:
def extract_regulated_genes(df):
    
    down_genes = df['gene'][df['direction'] == 'DOWN'].to_list()
    up_genes = df['gene'][df['direction'] == 'UP'].to_list()
    regulated_genes = pd.DataFrame((dict([ (k,pd.Series(v)) for k,v in {'gene' : df['gene'], 
                                                                        'direction' : df['direction'], 
                                                                        'valence' : df['logdiff']}.items() ])))
    
    regulated_genes = regulated_genes.sort_values(by = 'valence', ascending = False).set_index('gene')
    
    return regulated_genes

In [48]:
regulated_genes = extract_regulated_genes(df2)

#### 6) This is the function that actually generates the index, and automatically applies it to each sample in the dataset (each column) 

In [74]:
def generate_index(df, sample_cols):
    
    ## initialize the dataframe with the sample columns and create an index per sample array
    ## also, create an int counter
    samples = sample_cols
    index_per_sample = [0] * len(samples)
    i = -1
    
    ## iterate through the sampels and calculate the index given the index genes
    for sample in samples:
        i = i + 1
        index_per_sample[i] = np.mean(df[sample][df['direction'] == 'UP']) / np.mean(df[sample][df['direction'] == 'DOWN'])
        
    ## create the output dataframe by stitching this index per sample array to the corresponding sample names    
    final_df = pd.DataFrame([index_per_sample], columns = samples)
        
    return final_df

In [82]:
index = generate_index(df2, sample_cols = df2.columns[1:-4])

#### 7) In this final function, we are scaling all of the index values to be between 0 and 1.

In [83]:
def scale_index(df):

    ## scale data to between 0 and 1
    final_df_scaled = pd.DataFrame(df.iloc[0] - np.min(df.iloc[0])) / (np.max(df.iloc[0] - np.min(df.iloc[0])))
    final_df_scaled.reset_index(inplace = True)
    
    return final_df_scaled

In [84]:
index = scale_index(index)

In [85]:
index

,level_0,level_1,level_2,level_3,level_4,0
0,E18,Female,SAL,F_E18 1,Unnamed: 1_level_4,0.074341
1,E18,Male,SAL,M_E18 1,Unnamed: 2_level_4,0.039307
2,E18,Female,SAL,F_E18 3,Unnamed: 3_level_4,0.069697
3,E18,Male,SAL,M_E18 4,Unnamed: 4_level_4,0.000000
4,P14,Female,SAL,F_P14 1,Unnamed: 5_level_4,0.362481
5,P14,Female,SAL,F_P14 2,Unnamed: 6_level_4,0.356923
6,P14,Female,SAL,F_P14 3,Unnamed: 7_level_4,0.290847
7,P14,Female,SAL,F_P14 4,Unnamed: 8_level_4,0.382989
8,P14,Female,SAL,F_P14 5,Unnamed: 9_level_4,0.248252
9,P14,Female,SAL,F_P14 6,Unnamed: 10_level_4,0.336164
